In [1]:
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="llama2")

### Read transaction data

In [21]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv", sep=";")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2022-01-01,Pharmacy,Expense,45.64
1,2022-01-02,Gift,Income,1409.07
2,2022-01-03,Electronics,Expense,57.54
3,2022-01-05,Rent,Income,1117.50
4,2022-01-07,Supermarket,Expense,1885.56


In [4]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

39

In [5]:
unique_transactions[1:10]

array(['Gift', 'Electronics', 'Rent', 'Supermarket', 'Public Transport',
       'Audible Uk AdblCo/Pymt Gbr', 'Classpass* Monthly', 'Amazon Lux',
       'Bookstore'], dtype=object)

### Categorise bank transactions with Llama2

In [6]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 39]

In [7]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['Hello - World', 'Hello - there!'])

ResponseChecks(data=None)

In [8]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')


    response = response[1:] if response[0] == '' else response
    
    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index].strip() == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]


    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [9]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Etos AMSTERDAM NLD, Bistro Bar Amsterdam', llm)

['1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Vishandel Sier AMSTELVEEN - Groceries', '5. Etos AMSTERDAM NLD - Convenience Store', '6. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
1,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
2,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
3,4. Vishandel Sier AMSTELVEEN - Groceries,4. Vishandel Sier AMSTELVEEN,Groceries
4,5. Etos AMSTERDAM NLD - Convenience Store,5. Etos AMSTERDAM NLD,Convenience Store
5,6. Bistro Bar Amsterdam - Dining,6. Bistro Bar Amsterdam,Dining


In [10]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

['1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. Pharmacy - Personal Care', '4. Gift - Miscellaneous', '5. Electronics - Technology', '6. Rent - Housing', '7. Supermarket - Groceries', '8. Public Transport - Travel', '9. Audible Uk AdblCo/Pymt Gbr - Entertainment', '10. Classpass* Monthly - Fitness', '11. Amazon Lux - Online Shopping', '12. Bookstore - Books', '13. Online Shopping - Retail', '14. Coffee Shop - Food & Beverage', '15. Utility Bill - Bills & Utilities', '16. Pet Supplies - Pets', '17. Clothing Store - Fashion', '18. Cinema - Entertainment', '19. Salary - Income', '20. Freelance Work - Work', '21. Gym Membership - Fitness', '22. Restaurant - Food & Beverage', '23. Tesco - Groceries', '24. Apple Services - Technology', '25. Taxi Utrecht - Transportation', '26. Blogging - Writing', '27. Tikkie - Finance', '28. Freelancing - Work', '29. Bouldermuur Bv - Business', '30. Birtat Restaurant - Food & Beverage', '31. Beta Boulders Ams - S

In [11]:
categories_df_all.head()

,Transaction vs category,Transaction,Category
0,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
1,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
2,3. Pharmacy - Personal Care,3. Pharmacy,Personal Care
3,4. Gift - Miscellaneous,4. Gift,Miscellaneous
4,5. Electronics - Technology,5. Electronics,Technology


In [12]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Entertainment', 'Sport', 'Personal Care', 'Miscellaneous',
       'Technology', 'Housing', 'Groceries', 'Travel', 'Fitness',
       'Online Shopping', 'Books', 'Retail', 'Food & Beverage',
       'Bills & Utilities', 'Pets', 'Fashion', 'Income', 'Work',
       'Transportation', 'Writing', 'Finance', 'Business',
       'Sports & Fitness', 'Payment', 'Charity', 'Personal', 'Government'],
      dtype=object)

In [13]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [14]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'^\d+\.\s+', '', regex=True)
categories_df_all

,Transaction vs category,Transaction,Category
0,1. Spotify AB by Adyen - Entertainment,Spotify AB by Adyen,Entertainment
1,2. Beta Boulders Ams Amsterdam Nld - Sport,Beta Boulders Ams Amsterdam Nld,Sport and Fitness
2,3. Pharmacy - Personal Care,Pharmacy,Personal Care
3,4. Gift - Miscellaneous,Gift,Miscellaneous
4,5. Electronics - Technology,Electronics,Technology
5,6. Rent - Housing,Rent,Housing
6,7. Supermarket - Groceries,Supermarket,Groceries
7,8. Public Transport - Travel,Public Transport,Travel
8,9. Audible Uk AdblCo/Pymt Gbr - Entertainment,Audible Uk AdblCo/Pymt Gbr,Entertainment
9,10. Classpass* Monthly - Fitness,Classpass* Monthly,Fitness


In [24]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv", sep=";")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
# Очистка строк в обоих DataFrame
df['Name / Description'] = df['Name / Description'].str.strip().str.lower()
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.strip().str.lower()
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
filtered_df = df[df['Category'] == 'Travel']
filtered_df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
5,2022-01-07,public transport,Expense,117.84,8. Public Transport - Travel,public transport,Travel
6,2022-01-08,public transport,Expense,1148.68,8. Public Transport - Travel,public transport,Travel
7,2022-01-08,public transport,Expense,42.86,8. Public Transport - Travel,public transport,Travel
14,2022-01-22,public transport,Expense,1977.30,8. Public Transport - Travel,public transport,Travel
24,2022-02-09,public transport,Expense,101.63,8. Public Transport - Travel,public transport,Travel
64,2022-04-04,public transport,Expense,6.10,8. Public Transport - Travel,public transport,Travel
77,2022-04-29,public transport,Expense,102.96,8. Public Transport - Travel,public transport,Travel
78,2022-04-29,public transport,Expense,196.71,8. Public Transport - Travel,public transport,Travel
94,2022-05-26,public transport,Expense,1596.76,8. Public Transport - Travel,public transport,Travel
95,2022-05-26,public transport,Expense,10.49,8. Public Transport - Travel,public transport,Travel


In [25]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)